In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
token_path = '/content/drive/MyDrive/Minor_proj/captions1.txt'
text = open(token_path, 'r', encoding = 'utf-8').read()
print(text[:500])

1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set of stairs in an entry way .
1000268201_693b08cb0e.jpg,A girl going into a wooden building .
1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playhouse .
1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a wooden cabin .
1001773457_577c3a7d70.jpg,A black dog and a spotted dog are fighting
1001773457_577c3a7d70.jpg,A bl


In [5]:
def load_description(text):
    mapping = dict()
    for line in text.split("\n"):
        token = line.split(",")
        if len(line) < 2:
            continue
        img_id = token[0].split('.')[0]
        img_des = token[1]
        if img_id not in mapping:
            mapping[img_id] = list()
        mapping[img_id].append(img_des)
    return mapping

captions= load_description(text)
print("Number of items: " + str(len(captions)))

Number of items: 12


In [6]:
import string
def clean_description(desc):
    for key, des_list in desc.items():
        for i in range(len(des_list)):
            caption = des_list[i]
            caption = [ch for ch in caption if ch not in string.punctuation]
            caption = ''.join(caption)
            caption = caption.split(' ')
            caption = [word.lower() for word in caption if len(word)>1 and word.isalpha()]
            caption = ' '.join(caption)
            des_list[i] = caption

clean_description(captions)
captions['1000268201_693b08cb0e']

['child in pink dress is climbing up set of stairs in an entry way',
 'girl going into wooden building',
 'little girl climbing into wooden playhouse',
 'little girl climbing the stairs to her playhouse',
 'little girl in pink dress going into wooden cabin']

In [7]:
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.inception_v3 import preprocess_input

# Function to load and preprocess images
def load_images(directory):
    images = {}
    for filename in os.listdir(directory):
        img_id = filename.split('.')[0]  # Extract image id from filename
        img = load_img(os.path.join(directory, filename), target_size=(299, 299))
        img = img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)  # Preprocess image for InceptionV3
        images[img_id] = img
    return images

# Path to images directory
img_dir = '/content/drive/MyDrive/Minor_proj/images'

# Load images
img_features = load_images(img_dir)


In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Flatten captions into list
all_captions = [caption for captions_list in captions.values() for caption in captions_list]

# Create tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_captions)

# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1  # Plus one for padding (index 0)

# Convert captions to sequences of integers
sequences = tokenizer.texts_to_sequences(all_captions)

# Pad sequences to maximum length
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')


In [9]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, concatenate
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Load InceptionV3 as the image feature extractor
base_model = InceptionV3(weights='imagenet')
image_features = base_model.layers[-2].output

# Define LSTM to process captions
caption_input = Input(shape=(max_length,))
caption_embedding = Embedding(input_dim=vocab_size, output_dim=256, mask_zero=True)(caption_input)
caption_lstm = LSTM(256)(caption_embedding)

# Combine image and caption features
merged = concatenate([image_features, caption_lstm])
output_layer = Dense(vocab_size, activation='softmax')(merged)

# Create final model
model = Model(inputs=[base_model.input, caption_input], outputs=output_layer)

# Freeze InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')


96112376/96112376 [==============================] - 1s 0us/step


In [3]:
from google.colab import drive
drive.mount('/content/drive')

token_path = '/content/drive/MyDrive/Minor_proj/captions1.txt'

# Open and read the file
with open(token_path, 'r', encoding='utf-8') as file:
    text = file.read()

print(text[:500])

def load_description(text):
    mapping = dict()
    for line in text.split("\n"):
        token = line.split(",")
        if len(line) < 2:
            continue
        img_id = token[0].split('.')[0]
        img_des = token[1]
        if img_id not in mapping:
            mapping[img_id] = list()
        mapping[img_id].append(img_des)
    return mapping

captions = load_description(text)
print("Number of items: " + str(len(captions)))

import string
def clean_description(desc):
    for key, des_list in desc.items():
        for i in range(len(des_list)):
            caption = des_list[i]
            caption = [ch for ch in caption if ch not in string.punctuation]
            caption = ''.join(caption)
            caption = caption.split(' ')
            caption = [word.lower() for word in caption if len(word) > 1 and word.isalpha()]
            caption = ' '.join(caption)
            des_list[i] = caption

clean_description(captions)
print(captions['1000268201_693b08cb0e'])

import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.inception_v3 import preprocess_input

# Function to load and preprocess images
def load_images(directory):
    images = {}
    for filename in os.listdir(directory):
        img_id = filename.split('.')[0]  # Extract image id from filename
        img = load_img(os.path.join(directory, filename), target_size=(299, 299))
        img = img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)  # Preprocess image for InceptionV3
        images[img_id] = img
    return images

# Path to images directory
img_dir = '/content/drive/MyDrive/Minor_proj/images'

# Load images
img_features = load_images(img_dir)

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Flatten captions into list
all_captions = [caption for captions_list in captions.values() for caption in captions_list]

# Create tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_captions)

# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1  # Plus one for padding (index 0)

# Convert captions to sequences of integers
sequences = tokenizer.texts_to_sequences(all_captions)

# Pad sequences to maximum length
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, concatenate
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Load InceptionV3 as the image feature extractor
base_model = InceptionV3(weights='imagenet')
image_features = base_model.layers[-2].output

# Define LSTM to process captions
caption_input = Input(shape=(max_length,))
caption_embedding = Embedding(input_dim=vocab_size, output_dim=256, mask_zero=True)(caption_input)
caption_lstm = LSTM(256)(caption_embedding)

# Combine image and caption features
merged = concatenate([image_features, caption_lstm])
output_layer = Dense(vocab_size, activation='softmax')(merged)

# Create final model
model = Model(inputs=[base_model.input, caption_input], outputs=output_layer)

# Freeze InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Define the data generator function
def data_generator(img_features, captions, tokenizer, max_length, vocab_size, batch_size):
    keys = list(captions.keys())
    while True:
        for i in range(0, len(keys), batch_size):
            X1, X2, y = [], [], []
            for j in range(i, min(i + batch_size, len(keys))):
                img_id = keys[j]
                img_feature = img_features[img_id][0]  # Adjusted to access the image correctly
                caption_list = captions[img_id]
                for caption in caption_list:
                    seq = tokenizer.texts_to_sequences([caption])[0]
                    for k in range(1, len(seq)):
                        in_seq, out_word = seq[:k], seq[k]
                        in_seq = pad_sequences([in_seq], maxlen=max_length, padding='post')[0]
                        out_seq = np.zeros(vocab_size)
                        out_seq[out_word] = 1  # Corrected line
                        X1.append(img_feature)
                        X2.append(in_seq)
                        y.append(out_seq)
            yield [np.array(X1), np.array(X2)], np.array(y)

# Ensure batch_size is not larger than the number of images
batch_size = min(64, len(img_features))  # Adjust batch size accordingly

# Create the generator instance
train_data_generator = data_generator(img_features, captions, tokenizer, max_length, vocab_size, batch_size)

# Define the number of epochs and steps per epoch
epochs = 10
steps_per_epoch = len(img_features) // batch_size

# Ensure steps_per_epoch is at least 1
steps_per_epoch = max(steps_per_epoch, 1)

# Fit the model
model.fit(train_data_generator, epochs=epochs, steps_per_epoch=steps_per_epoch, verbose=1)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set of stairs in an entry way .
1000268201_693b08cb0e.jpg,A girl going into a wooden building .
1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playhouse .
1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a wooden cabin .
1001773457_577c3a7d70.jpg,A black dog and a spotted dog are fighting
1001773457_577c3a7d70.jpg,A bl
Number of items: 12
['child in pink dress is climbing up set of stairs in an entry way', 'girl going into wooden building', 'little girl climbing into wooden playhouse', 'little girl climbing the stairs to her playhouse', 'little girl in pink dress going into wooden cabin']
Epoch 1/10
1/1 [==============================] - 63s 63s/step - loss: 5.3203
Epoch 2/10
1/1 [=

In [18]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-10) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_caption(model, tokenizer, image, max_length, temperature=1.0):
    in_text = 'startseq'
    for _ in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        preds = model.predict([image, sequence], verbose=0)[0]
        yhat = sample(preds, temperature)
        word = tokenizer.index_word.get(yhat, None)
        if word is None or word == 'endseq':
            break
        in_text += ' ' + word
    return in_text

# Load and preprocess the image (replace 'image_path' with the actual image file path)
from tensorflow.keras.preprocessing.image import load_img, img_to_array
image_path = '/content/drive/MyDrive/Minor_proj/images/1000268201_693b08cb0e.jpg'
image = load_img(image_path, target_size=(299, 299))
image = img_to_array(image)
image = np.expand_dims(image, axis=0)
image = preprocess_input(image)

# Generate a caption for the image
caption = generate_caption(model, tokenizer, image, max_length, temperature=1.0)
print('Generated Caption:', caption)

Generated Caption: startseq at dress dress wooden her wooden of an dress dress stairs pink an an dress wooden set into the
